# DataArray Schemas 

Use the {py:class}`~.DataArraySchema` to specify schema that validate {py:attr}`xarray.DataArray` objects.

The {py:class}`~.DataArraySchema` object is composed of optional sub-schemas (termed *components*), each specifying a different feature of the array.

The following components are currently supported: 

- {py:class}`~.AttrsSchema`;
- {py:class}`~.CoordsSchema`;
- {py:class}`~.DimsSchema`;
- {py:class}`~.DTypeSchema`;
- {py:class}`~.NameSchema` .
- {py:class}`~.ShapeSchema`;

In [1]:
import xarray_jsonschema as xj

schema = xj.DataArraySchema(
    name='foo',
    dims=('dim0', xj.NameSchema('^dim[0-9]$', regex=True)),
    dtype=float,
)

To validate an {py:class}`~xarray.DataArray` against the schema, you can
use the {py:meth}`~.DataArraySchema.validate` method, or, call the schema
object directly (which calls {py:meth}`~.DataArraySchema.validate` under the hood):

In [2]:
good_da = xr.DataArray(name='foo')
schema(good_da)

ds = good_da.to_dataset()
ds.shape

NameError: name 'xr' is not defined

If nothing happends, validation was successful. If validation fails, a {py:class}`jsonschema.exceptions.ValidationError` exception is raised.

In [6]:
bad_da = xr.DataArray(name='bar')

try:
    schema(bad_da)
except xjs.ValidationError as error:
    print(error.message)

'foo' was expected


The `coords` parameter accepts a mapping of {py:class}`~.DataArraySchema` objects. This makes it easy to create reusable schema that you can pass around.

In [27]:
x = xjs.DataArraySchema(name='x', dtype=int)
y = xjs.DataArraySchema(name='y', dtype=float)
xy_schema = xjs.DataArraySchema(coords={'x': x, 'y': y})

xy_da = xr.DataArray(
    dims=('x', 'y'),
    coords=[[1, 2], [3.0, 4.0]],
)
xy_schema(xy_da)


You can view the JSON Schema by calling the {py:attr}`~.DataArraySchema.json` property.

In [28]:
xy_schema.json

{'type': 'object',
 'properties': {'coords': {'type': 'object',
   'properties': {'x': {'type': 'object',
     'properties': {'dtype': {'const': 'int64'}, 'name': {'const': 'x'}}},
    'y': {'type': 'object',
     'properties': {'dtype': {'const': 'float64'}, 'name': {'const': 'y'}}}},
   'additionalProperties': True,
   'required': ['x', 'y']}}}